# NLP con Hugging Face

## Procesando los datos para NLP

### Descargando el dataset

In [7]:
from datasets import load_dataset

ds = load_dataset('glue', 'mrpc')

Usaremos el dataset MRPC. Este es uno de los 10 datasets que componen el [benchmark (punto de referencia) GLUE](https://huggingface.co/datasets/glue). Se utiliza para medir el rendimiento de los modelos ML en 10 tareas de clasificación de texto diferentes.

En otras palabras, seleccionamos el subset `mrpc` del dataset `glue`:

In [8]:
ex = ds['train'][0]
print(ex)

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', 'label': 1, 'idx': 0}


In [9]:
# type of ex is dict
print(ds['train'])

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 3668
})


Así se ve un ejemplo. Notamos que `mrpc` está compuesto de dos oraciones y una etiqueta que indica si los dos enunciados son equivalentes.

In [10]:
labels = ds['train'].features['label']

In [11]:
labels.int2str(0)

'not_equivalent'

### Tokenizando

¿Recuerdas que con visión descargamos el feature extractor directamente del repositorio del modelo pre-entrenado que vamos a usar como base?

Podemos pensar en la función tokenizadora como el equivalente en el NLP.

Descargamos el tokenizador directamente del repo del modelo que usaremos.

In [12]:
from transformers import AutoTokenizer

repo_id = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(repo_id)

Para preprocesar el conjunto de datos necesitamos convertir el texto en números que el modelo pueda entender. Esto se hace con un tokenizador. 

Pasar de texto a números se conoce como codificación o encoding. El encoding se realiza en un proceso de dos pasos: la tokenización, seguida de la conversión a input ids. Por el momento nos basta saber que estamos traduciendo texto a números llamados como input ids. Estos estarán en el formato adecuado para alimentar nuestro modelo.

Podemos alimentar al tokenizador con una oración o una lista de oraciones, por lo que podemos tokenizar directamente todas las primeras oraciones y todas las segundas oraciones de cada par de esta manera:

In [13]:
tokenized_sentece_1 = tokenizer(ds['train']['sentence1'][2])
tokenized_sentece_1

{'input_ids': [101, 2027, 2018, 2405, 2019, 15147, 2006, 1996, 4274, 2006, 2238, 2184, 1010, 5378, 1996, 6636, 2005, 5096, 1010, 2002, 2794, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Necesitamos manejar los dos enunciados como un par y no separados. El tokenizador puede tomar un par de secuencias y prepararlas de la manera que espera nuestro modelo:

In [14]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

¿Qué significa cada uno de los valores que nos retorna el tokenizador?
- `input_ids` es la traducción de palabras a números.
- `attention_mask` es un tensor con la misma forma que `input_ids`, pero lleno de 0 y 1: los 1 indican que se debe atender a los tokens correspondientes y los 0 indican que no se deben atender. Es decir, deben ser ignorados por el modelo.
- `token_type_ids` dice al modelo qué parte de la entrada es la primera oración y cuál es la segunda oración.

El modelo espera que las entradas sean de la forma [CLS] oración 1 [SEP] oración 2 [SEP] cuando hay dos oraciones.

In [15]:
tokenizer.convert_ids_to_tokens(inputs['input_ids'])

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 'sentence',
 '.',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 'one',
 '.',
 '[SEP]']

Si seleccionamos otro modelo en el Hub no necesariamente tendremos `token_type_ids` en las entradas tokenizadas (por ejemplo, no se devuelven si usa un modelo `DistilBERT`). Solo se devuelven cuando el modelo sabrá qué hacer con ellas, porque los ha visto durante su preentrenamiento.

En general, no necesitamos preocuparnos por si hay o no `token_type_ids` en nuestras entradas tokenizadas, siempre que usemos el tokenizador correspondiente al modelo, todo estará bien ya que el tokenizador sabe qué proporcionar al modelo.

Por ejemplo, durante esta clase utilizaremos un modelo [`distilroberta-base`](https://huggingface.co/distilroberta-base) por su tamaño y efectividad. Pero no cuenta con `token_type_ids` y aún así nos regresa excelentes resultados.

En la organización del Platzi en el Hub puedes encontrar un [modelo BERT](https://huggingface.co/platzi/platzi-distilroberta-base-mrpc-glue-omar-espejel) afinado siguiendo el mismo proceso que usamos en esta clase.

In [16]:
repo_id = "distilroberta-base"

tokenizer = AutoTokenizer.from_pretrained(repo_id)

Creamos una función tokenizadora. Recibe un ejemplo y lo tokeniza.

In [17]:
def tokenize_fn(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True)

In [18]:
prepared_ds = ds.map(tokenize_fn, batched=True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

### Definiendo el data collator: Dynamic padding

Necesitamos que nuestros tensores tengan una forma rectangular. Es decir que tengan el mismo tamaño cada uno de los ejemplos. Sin embargo, los textos no necesariamente tienen el mismo tamaño. 

Para ello usamos el relleno o padding. El padding se asegura de que todas nuestras oraciones tengan la misma longitud al agregar una palabra especial llamada padding token a las oraciones con menos valores. Por ejemplo, si tenemos 10 oraciones con 10 palabras y 1 oración con 20 palabras, el relleno garantizará que todas las oraciones tengan 20 palabras.

Dejamos el argumento de `padding` del tokenizer vacío en nuestra función de tokenización por ahora. Esto se debe a que rellenar (hacer padding) todas las muestras hasta la longitud máxima del dataset no es eficiente, es mejor rellenar las muestras cuando estamos construyendo un batch, ya que entonces solo necesitamos rellenar hasta la longitud máxima en ese batch, y no la longitud máxima en todo el dataset. ¡Esto puede ahorrar mucho tiempo y potencia de procesamiento cuando las entradas tienen longitudes muy variables!

Usaremos un DataCollator para esto.

Rellenemos (hagamos padding) todos los ejemplos con la longitud del elemento más largo del batch. A esta técnica se le conoce como relleno dinámico o dynamic padding.

In [19]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Entrenamiento y evaluación

Definamos el resto de los argumentos necesarios para `Trainer`.

### Definiendo la métrica 

In [20]:
import evaluate
import numpy as np

def compute_metrics(eval_pred):
  metric = evaluate.load("glue", "mrpc")
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

### Configurando `Trainer`


In [21]:
from transformers import AutoModelForSequenceClassification

labels = ds['train'].features['label'].names

model = AutoModelForSequenceClassification.from_pretrained(
  repo_id, 
  num_labels=len(labels), 
  id2label={str(i): label for i, label in enumerate(labels)}, 
  label2id={label: i for i, label in enumerate(labels)}
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir='./transfer-course-distilroberta-base-mrpc-glue-nestor-mamani',          # output directory
  num_train_epochs=3,              # total number of training epochs
  per_device_train_batch_size=16,  # batch size per device during training
  per_device_eval_batch_size=64,   # batch size for evaluation
  warmup_steps=500,                # number of warmup steps for learning rate scheduler
  weight_decay=0.01,               # strength of weight decay
  logging_dir='./logs',            # directory for storing logs
  logging_steps=10,
  evaluation_strategy='steps',
  eval_steps=500,
  load_best_model_at_end=True,
  push_to_hub=True,
  push_to_hub_organization="platzi"
)

/Users/nestorxyz/Library/Caches/pypoetry/virtualenvs/similarity-search-101-r2xOf8wD-py3.9/lib/python3.9/site-packages/transformers/training_args.py:1738: FutureWarning: `--push_to_hub_organization` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case platzi/transfer-course-distilroberta-base-mrpc-glue-nestor-mamani).
  warnings.warn(


In [23]:
from huggingface_hub import notebook_login

notebook_login()

In [25]:
from transformers import Trainer

trainer = Trainer(
  model=model,                         # the instantiated 🤗 Transformers model to be trained
  args=training_args,                  # training arguments, defined above
  train_dataset=prepared_ds['train'],         # training dataset
  eval_dataset=prepared_ds['validation'],             # evaluation dataset
  data_collator=data_collator,
  tokenizer=tokenizer,
  compute_metrics=compute_metrics,
)

### Entrenamiento

In [26]:
train_result = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_result.metrics)
trainer.save_metrics("train", train_result.metrics)

  0%|          | 0/690 [00:00<?, ?it/s]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.7001, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.04}
{'loss': 0.6929, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.09}
{'loss': 0.6827, 'learning_rate': 3e-06, 'epoch': 0.13}
{'loss': 0.6752, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.17}
{'loss': 0.657, 'learning_rate': 5e-06, 'epoch': 0.22}
{'loss': 0.6256, 'learning_rate': 6e-06, 'epoch': 0.26}
{'loss': 0.6543, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.3}
{'loss': 0.6606, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.35}
{'loss': 0.6093, 'learning_rate': 9e-06, 'epoch': 0.39}
{'loss': 0.5816, 'learning_rate': 1e-05, 'epoch': 0.43}
{'loss': 0.6384, 'learning_rate': 1.1000000000000001e-05, 'epoch': 0.48}
{'loss': 0.5593, 'learning_rate': 1.2e-05, 'epoch': 0.52}
{'loss': 0.6282, 'learning_rate': 1.3000000000000001e-05, 'epoch': 0.57}
{'loss': 0.6028, 'learning_rate': 1.4000000000000001e-05, 'epoch': 0.61}
{'loss': 0.5705, 'learning_rate': 1.5e-05, 'epoch': 0.65}
{'loss': 0.489, 'learning

  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.46014174818992615, 'eval_accuracy': 0.8357843137254902, 'eval_f1': 0.8858603066439524, 'eval_runtime': 2.9507, 'eval_samples_per_second': 138.271, 'eval_steps_per_second': 2.372, 'epoch': 2.17}
{'loss': 0.2844, 'learning_rate': 4.736842105263158e-05, 'epoch': 2.22}
{'loss': 0.1503, 'learning_rate': 4.473684210526316e-05, 'epoch': 2.26}
{'loss': 0.3402, 'learning_rate': 4.210526315789474e-05, 'epoch': 2.3}
{'loss': 0.279, 'learning_rate': 3.9473684210526316e-05, 'epoch': 2.35}
{'loss': 0.3684, 'learning_rate': 3.6842105263157895e-05, 'epoch': 2.39}
{'loss': 0.3793, 'learning_rate': 3.421052631578947e-05, 'epoch': 2.43}
{'loss': 0.2879, 'learning_rate': 3.157894736842105e-05, 'epoch': 2.48}
{'loss': 0.3529, 'learning_rate': 2.8947368421052634e-05, 'epoch': 2.52}
{'loss': 0.2574, 'learning_rate': 2.6315789473684212e-05, 'epoch': 2.57}
{'loss': 0.4201, 'learning_rate': 2.368421052631579e-05, 'epoch': 2.61}
{'loss': 0.2821, 'learning_rate': 2.105263157894737e-05, 'epoch': 2.

### Evaluación

In [27]:
metrics = trainer.evaluate(prepared_ds['validation'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

  0%|          | 0/7 [00:00<?, ?it/s]

***** eval metrics *****
  epoch                   =        3.0
  eval_accuracy           =     0.8358
  eval_f1                 =     0.8859
  eval_loss               =     0.4601
  eval_runtime            = 0:00:01.84
  eval_samples_per_second =    220.794
  eval_steps_per_second   =      3.788


### Compartimos en el Hub

False